In [40]:
import pandas as pd
import numpy as np
import collections, re
import string
from copy import deepcopy

In [41]:
f = "imdb_data.csv"
df = pd.DataFrame.from_csv(f)
df

,Review,Score
1,"A very, very, very slow-moving, aimless movie ...",0
2,Not sure who was more lost - the flat characte...,0
3,Attempting artiness with black & white and cle...,0
4,Very little music or anything to speak of.,0
5,The best scene in the movie was when Gerardo i...,1
6,"The rest of the movie lacks art, charm, meanin...",0
7,Wasted two hours.,0
8,Saw the movie today and thought it was a good ...,1
9,A bit predictable.,0
10,Loved the casting of Jimmy Buffet as the scien...,1


In [42]:
sentences = list(df["Review"])
all_words = dict()
for sentence in sentences:
    for word in [word.strip(string.punctuation) for word in sentence.split()]:
        all_words[word.lower()] = 0
word_vectors = {}
key = 0

for sentence in sentences:
    word_vectors[key] = deepcopy(all_words)
    for word in [word.strip(string.punctuation) for word in sentence.split()]:
        word_vectors[key][word.lower()] += 1
    key += 1

In [50]:
vector1 = np.array([[1,2,3]]) # not really sure how to represent a review
vector2 = np.array([[1,-2,7]])

lamb = 5 # need to get this with cross validation

In [51]:
"""
Represents a similarity model between two 
instances that returns a value from [0, 1].
"""
def sim_measure(x, y):
    diff = x - y
    norm_squared = (np.linalg.norm(diff))**2
    return np.exp(-norm_squared)

In [52]:
"""
A squared loss penalty function that returns the 
squared difference between the prediciton of instance x
and the prediction of instance y.
"""
def penalty_one(x, y, predictor):
    return (predictor(x) - predictor(y))**2

In [53]:
"""
A squared loss penalty function that returns the 
squared difference between the prediciton and the
actual label.
"""
def penalty_two(instance, label, predictor):
    return (predictor(x) - label)**2

In [54]:
"""
A loss function that averages the labels of each of 
its instances.
"""
def loss_function(predictor, group):
    score = 0
    for instance in group:
        score += predictor(instance)
    return float(score) / (len(group))

In [55]:
"""
The cost funciton that looks at all instances and groups
and applies the predictor to it.
"""
def cost_function(instances, groups, predictor):
    score1 = 0
    for i in range(instances):
        for j in range(instances):
            score1 += sim_measure(i, j)*penalty(i, j, predictor)
    score1 = float(score) / len(instances)
    
    score2 = 0
    for group in groups:
        instance = group[0]
        label = group[1]
        score2 += penalty_two(instance, label, predictor)
    score2 = float(score2) * (lamb / len(groups))
    
    return score1 + score2